In [43]:
from cylib.qmtdata.cyxtdata import xtdata  # QMT
from cylib.apis.all_api import *
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import date, datetime

- Date duration: 20180101-20231031
- Filter condition
  - Market time > 3M
  - Balances of CB >= 200m
  - Credit >= A+
  - Remove ST and *ST
- Position Adjustment: Monthly
  - Change CB pool at the beginning of month

In [51]:
def Today_Date():
    today_date = date.today().strftime('%Y%m%d')
    return today_date
def Trade_Calendar(Begin_date, End_date, Asset_Type='index'):
    """
    Stay in CY environment: from cylib.apis.all_api import *
    Type: 'stock' or 'convertbond'
    """
    PRICE = get_price(
        ts_code_list=["000300.SH"],
        feature_list=["open"],
        start_date=Begin_date,
        trade_date=End_date,
        target_type=Asset_Type,
    )
    PRICE.reset_index(inplace=True)
    PRICE = PRICE.sort_values(by="trade_date").reset_index(drop=True)
    PRICE_pivot = PRICE.fillna(0)
    PRICE_pivot = PRICE_pivot.pivot(index="trade_date", columns="ts_code", values="open")

    # Attention: len(date_all) >= 60
    date_all = PRICE_pivot.index
    return date_all

def Position_Changing_Date(Trade_Date, Frequency='Weekly', Times=1):
    """
    Frequency: 'Times' Week(s) or 'Times' Month(s)
    """
    # Trade_Date = Trade_Calendar(Begin_date, End_date)
    # dates = [datetime for date_str in Trade_Date]
    
    # Sort the date list in ascending order
    sorted_dates = sorted(Trade_Date)
    
    Position_dates = []
    if Frequency == 'Monthly':
        current_month = date[0].month
        Position_dates.append(date[0])
        for date in sorted_dates:
            # Get the month of the current date
            date_month = date.month
            # Check if it's a new month
            if date_month != current_month:
                Position_dates.append(date)
                current_month = date_month
    elif Frequency == 'Weekly':
        current_week = date[0].week
        Position_dates.append(date[0])
        for date in sorted_dates:
            date_week = date.week
            # Check if it's a new week
            if date_week != current_week:
                Position_dates.append(date)
                current_week = date_week
    Position_dates = Position_dates[::Times]
    return Position_dates

def Find_Missing_Items(Price_DF, Asset_list):
    """
    Finding the missing items in price dataframe.
    Asset_List: The asset list which is the pool of all assets
    """
    PRICE_pivot = Price_DF.pivot(index="trade_date", columns="ts_code", values="OPEN")
    Date_all = PRICE_pivot.index
    
    # Find missing items
    all_combinations = pd.MultiIndex.from_product(
        [Date_all, Asset_list], 
        names=["trade_date", "ts_code"]
    )
    all_combinations_df = pd.DataFrame(index=all_combinations).reset_index()
    
    # Left join
    price_whole = pd.merge(
        all_combinations_df, Price_DF, 
        on=["trade_date", "ts_code"], how="left"
    )
    missing = price_whole[price_whole.isnull().any(axis=1)]
    missing_stock = missing["ts_code"].unique()
    print("The missing items:", missing_stock)

    # # Remove the stocks cause they have missing values.
    # mask = all_stocks["ts_code"].isin(missing_stock)
    # # removing the filtered rows from the original dataframe
    # all_stocks = all_stocks[~mask]
    # ts_code = list(all_stocks["ts_code"])

In [53]:
begin_date = "20180101"
end_date = "20231031"
today_date = Today_Date()
Trade_date = Trade_Calendar(begin_date, end_date, Asset_Type='convertbond')
Trade_date

DatetimeIndex(['2018-01-02', '2018-01-03', '2018-01-04', '2018-01-05',
               '2018-01-08', '2018-01-09', '2018-01-10', '2018-01-11',
               '2018-01-12', '2018-01-15',
               ...
               '2023-10-18', '2023-10-19', '2023-10-20', '2023-10-23',
               '2023-10-24', '2023-10-25', '2023-10-26', '2023-10-27',
               '2023-10-30', '2023-10-31'],
              dtype='datetime64[ns]', name='trade_date', length=1414, freq=None)

In [54]:
Position_Changing_Date(Trade_date)

[Timestamp('2018-01-02 00:00:00'),
 Timestamp('2018-02-01 00:00:00'),
 Timestamp('2018-03-01 00:00:00'),
 Timestamp('2018-04-02 00:00:00'),
 Timestamp('2018-05-02 00:00:00'),
 Timestamp('2018-06-01 00:00:00'),
 Timestamp('2018-07-02 00:00:00'),
 Timestamp('2018-08-01 00:00:00'),
 Timestamp('2018-09-03 00:00:00'),
 Timestamp('2018-10-08 00:00:00'),
 Timestamp('2018-11-01 00:00:00'),
 Timestamp('2018-12-03 00:00:00'),
 Timestamp('2019-01-02 00:00:00'),
 Timestamp('2019-02-01 00:00:00'),
 Timestamp('2019-03-01 00:00:00'),
 Timestamp('2019-04-01 00:00:00'),
 Timestamp('2019-05-06 00:00:00'),
 Timestamp('2019-06-03 00:00:00'),
 Timestamp('2019-07-01 00:00:00'),
 Timestamp('2019-08-01 00:00:00'),
 Timestamp('2019-09-02 00:00:00'),
 Timestamp('2019-10-08 00:00:00'),
 Timestamp('2019-11-01 00:00:00'),
 Timestamp('2019-12-02 00:00:00'),
 Timestamp('2020-01-02 00:00:00'),
 Timestamp('2020-02-03 00:00:00'),
 Timestamp('2020-03-02 00:00:00'),
 Timestamp('2020-04-01 00:00:00'),
 Timestamp('2020-05-

In [19]:
CB = get_price(
    ts_code_list=[],
    feature_list=["open", "close", "low", "high"],
    start_date=begin_date,
    trade_date=end_date,
    back_len=10,
    target_type="convertbond",
)
CB.reset_index(inplace=True)
CB = CB.sort_values(by="trade_date").reset_index(drop=True)
# price.set_index(['trade_date', 'ts_code'], inplace=True)
# CB_pivot = CB.fillna(0)
# CB_pivot = CB_pivot.pivot(index="trade_date", columns="ts_code", values="open")
CB

,trade_date,ts_code,open,close,low,high
0,2023-08-08,128144.SZ,116.87,117.38,116.60,117.50
1,2023-08-08,113647.SH,123.02,123.38,122.85,124.20
2,2023-08-08,113648.SH,139.42,138.70,136.50,139.42
3,2023-08-08,113649.SH,126.47,125.88,125.65,126.80
4,2023-08-08,113650.SH,111.89,111.80,111.58,111.89
...,...,...,...,...,...,...
52972,2023-12-29,123197.SZ,130.68,133.05,130.60,133.85
52973,2023-12-29,123196.SZ,113.99,114.85,113.60,115.18
52974,2023-12-29,123195.SZ,122.66,122.99,122.30,124.10
52975,2023-12-29,123201.SZ,132.10,134.25,132.00,134.50


- **ts_code**: 可转债的唯一代码。
- **bond_full_name**: 可转债的完整名称。
- **bond_short_name**: 可转债的简称。
- **stk_code**: 可转债关联的股票代码。
- **stk_short_name**: 可转债关联的股票简称。
- **maturity**: 可转债的到期年限（年为单位）。
- **par**: 可转债的面值。
- **issue_price**: 可转债的发行价格。
- **issue_size**: 可转债的发行规模。
- **remain_size**: 可转债的剩余规模。
- **value_date**: 可转债起息日。
- **maturity_date**: 可转债到期日。
- **rate_type**: 利率类型，可能包括固定利率或者浮动利率。
- **coupon_rate**: 票面利率，即可转债每年支付的利息。
- **add_rate**: 附加利率，如果有的话，会增加到票面利率上。
- **pay_per_year**: 可转债每年支付利息的次数。
- **list_date**: 可转债上市日期。
- **delist_date**: 可转债退市日期。
- **exchange**: 可转债交易所代码。
- **conv_start_date**: 可转债的转换起始日期。
- **conv_end_date**: 可转债的转换结束日期。
- **first_conv_price**: 可转债的首次转换价格。
- **conv_price**: 可转债当前转换价格。
- **rate_clause**: 利率条款的详细说明，可能包括任何特殊的利率计算或者支付规则。

In [8]:
CB_INFO = get_targets_info(target_type='cb')
CB_INFO

,ts_code,bond_full_name,bond_short_name,stk_code,stk_short_name,maturity,par,issue_price,issue_size,remain_size,...,add_rate,pay_per_year,list_date,delist_date,exchange,conv_start_date,conv_end_date,first_conv_price,conv_price,rate_clause
0,125002.SZ,万科企业股份有限公司可转换公司债券,万科转债,000002.SZ,万科A,5.0,100.0,100.0,1.500000e+09,0.0,...,0.0,1,2002-06-28,2004-04-30,SZ,2002-12-13,2007-06-12,12.10,5.850,票面利率为每年1.5%
1,125009.SZ,中国宝安集团股份有限公司可转换公司债券,宝安转券,000009.SZ,中国宝安,3.0,1.0,NaN,5.000000e+08,0.0,...,NaN,1,1993-02-10,1996-01-01,SZ,1993-06-01,1995-12-31,25.00,19.392,3%
2,125069.SZ,深圳华侨城控股股份有限公司可转换公司债券,侨城转债,000069.SZ,华侨城A,3.0,100.0,100.0,4.000000e+08,0.0,...,NaN,1,2004-01-16,2005-04-29,SZ,2004-07-01,2006-12-30,6.15,6.150,"本次发行的可转换公司债券按票面金额计算利息,计息起始日为可转换公司债券发行首日,票面利率第一..."
3,125301.SZ,吴江丝绸股份有限公司可转换公司债券,丝绸转债,000301.SZ,东方盛虹,5.0,100.0,100.0,2.000000e+08,0.0,...,NaN,1,1998-09-15,2003-08-28,SZ,2000-05-29,2003-08-27,4.10,4.100,"可转债按票面金额由1998年8月28日起开始计算利息,可转债发行首年票面利率为1.00%,以..."
4,126301.SZ,吴江丝绸股份有限公司可转换公司债券,丝绸转2,000301.SZ,东方盛虹,5.0,100.0,100.0,8.000000e+08,0.0,...,NaN,1,2002-09-24,2006-09-18,SZ,2003-03-10,2007-09-08,8.78,3.000,本次可转换公司债券的年利率为1.8%。
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1007,118048.SH,广东利扬芯片测试股份有限公司向不特定对象发行可转换公司债券,利扬转债,688135.SH,利扬芯片,6.0,100.0,100.0,5.200000e+08,520000000.0,...,12.5,1,None,None,SH,2025-01-08,2030-07-01,16.13,16.130,"20240702-20250701,票面利率:0.2%;20250702-20260701,..."
1008,123241.SZ,深圳欧陆通电子股份有限公司向不特定对象发行可转换公司债券,欧通转债,300870.SZ,欧陆通,6.0,100.0,100.0,6.445300e+08,644530000.0,...,10.0,1,None,None,SZ,2025-01-11,2030-07-04,44.86,44.860,"20240705-20250704,票面利率:0.10%;20250705-20260704..."
1009,123242.SZ,广州市聚赛龙工程塑料股份有限公司向不特定对象发行可转换公司债券,赛龙转债,301131.SZ,聚赛龙,6.0,100.0,100.0,2.500000e+08,250000000.0,...,12.2,1,None,None,SZ,2025-01-12,2030-07-07,36.81,36.810,"20240708-20250707,票面利率:0.30%;20250708-20260707..."
1010,123243.SZ,浙江严牌过滤技术股份有限公司向不特定对象发行可转换公司债券,严牌转债,301081.SZ,严牌股份,6.0,100.0,100.0,4.678900e+08,467890000.0,...,11.5,1,None,None,SZ,2025-01-16,2030-07-09,7.58,7.580,"20240710-20250709,票面利率:0.20%;20250710-20260709..."


| 字段                     | 类型    | 说明                         |
|-------------------------|---------|------------------------------|
| bondCode                | str     | 可转债代码                   |
| bondName                | str     | 可转债简称                   |
| stockCode               | str     | 正股代码                     |
| stockName               | str     | 正股简称                     |
| bondMaturity            | float   | 发行年限                     |
| bondParvalue            | float   | 面值                         |
| bondIssuePrice          | float   | 发行价格                     |
| bondIssueSize           | float   | 发行总额（元）               |
| bondReMainSize          | float   | 债券余额（元）               |
| bondValueDate           | int     | 起息日期                     |
| bondMaturityDate        | int     | 到期日期                     |
| bondRateType            | str     | 利率类型                     |
| bondCouponRate          | float   | 票面利率                     |
| bondAddRate             | float   | 补偿利率                     |
| bondPayPerYear          | int     | 年付息次数                   |
| bondListDate            | int     | 上市日期                     |
| delistDate              | int     | 摘牌日                       |
| bondExchange            | str     | 上市地点                     |
| convStartDate           | int     | 转股起始日，转股日           |
| convEndDate             | int     | 转股截止日                   |
| firstConvPrice          | float   | 初始转股价                   |
| bondConvPrice           | float   | 最新转股价                   |
| rateClause              | str     | 利率说明                     |
| forceRedeemTradeDate    | int     | 强赎最后交易日               |
| forceRedeemConvDate     | int     | 强赎最后转股日               |
| forceRedeemPrice        | float   | 强赎价格                     |
| triggerForceRedeemPrice | float   | 强赎触发价                   |
| triggerRepurchasePrice  | float   | 回售触发价                   |
| expireRedeemPrice       | float   | 到期赎回价                   |
| analYTM                 | float   | 纯债YTM(%)                   |
| analPTM                 | float   | 剩余期限                     |
| analAccruedinterst      | float   | 应计利息                     |
| analStrbvalue           | float   | 纯债价值                     |
| analStrbpremium         | float   | 纯债溢价率(%)                |
| analConvvalue           | float   | 转股价值                     |
| analConvpremiumratio    | float   | 转股溢价率(%)                |
| analDuration            | float   | 久期                         |
| analConvexity           | float   | 凸性                         |
| infoProvisiontype       | str     | 条款类型，多种类型时使用 "," 分隔 |
| termYear                | float   | 债券期限（年）               |
| interestType            | int     | 付息利率品种,1-浮动利率，2-固定利率，3-累进利率 |
| level                   | str     | 债券评级                     |
| forceRedeemPriceRatio   | float   | 强赎触发比(%)                |
| redeemTerms             | str     | 强赎条款                     |
| nextPutDate             | int     | 回售起始日                   |
| putPrice                | float   | 回售价                       |
| putConvertPriceRatio    | float   | 回售触发比(%)                |
| putTerms                | str     | 回售条款                     |
| triggerAdjustPrice      | float   | 下修触发价                   |
| adjustPriceRatio        | float   | 下修触发比(%)                |
| adjustTerms             | str     | 下修条款                     |
| ytm                     | float   | 到期收益率(%)                |
| redeemStatus            | str     | 强赎状态                     |
| adjustStatus            | str     | 下修状态                     |
| readjustDate            | int     | 下修重算起始日               |
| regProvince             | str     | 所属省份                     |


In [34]:
# 下载转债信息
xtdata.download_cb_data()

In [63]:
xtdata.get_cb_info('128013.SZ')

{'bondCode': '',
 'bondName': '',
 'stockCode': '',
 'stockName': '',
 'bondMaturity': 0.0,
 'bondParvalue': 0.0,
 'bondIssuePrice': 0.0,
 'bondIssueSize': 0.0,
 'bondReMainSize': 0.0,
 'bondValueDate': 0,
 'bondMaturityDate': 0,
 'bondRateType': '',
 'bondCouponRate': 0.0,
 'bondAddRate': 0.0,
 'bondPayPerYear': 0,
 'bondListDate': 0,
 'delistDate': 0,
 'bondExchange': '',
 'convStartDate': 0,
 'convEndDate': 0,
 'firstConvPrice': 0.0,
 'bondConvPrice': 0.0,
 'rateClause': '',
 'forceRedeemTradeDate': 0,
 'forceRedeemConvDate': 0,
 'forceRedeemPrice': 0.0,
 'triggerForceRedeemPrice': 0.0,
 'triggerRepurchasePrice': 0.0,
 'expireRedeemPrice': 0.0,
 'analYTM': 0.0,
 'analPTM': 0.0,
 'analAccruedinterst': 0.0,
 'analStrbvalue': 0.0,
 'analStrbpremium': 0.0,
 'analConvvalue': 0.0,
 'analConvpremiumratio': 0.0,
 'analDuration': 0.0,
 'analConvexity': 0.0,
 'infoProvisiontype': '',
 'termYear': 0.0,
 'interestType': 0,
 'level': '',
 'forceRedeemPriceRatio': 0.0,
 'redeemTerms': '',
 'nextP

In [36]:
# 获取转债信息
cb_info = xtdata.get_cb_info("123219.SZ")
cb_info

{'bondCode': '123219.SZ',
 'bondName': '宇瞳转债',
 'stockCode': '300790.SZ',
 'stockName': '宇瞳光学',
 'bondMaturity': 6.0,
 'bondParvalue': 100.0,
 'bondIssuePrice': 100.0,
 'bondIssueSize': 600000000.0,
 'bondReMainSize': 599979900.0,
 'bondValueDate': 20230811,
 'bondMaturityDate': 20290811,
 'bondRateType': '',
 'bondCouponRate': 0.3,
 'bondAddRate': 0.0,
 'bondPayPerYear': 0,
 'bondListDate': 20230829,
 'delistDate': 20290810,
 'bondExchange': '',
 'convStartDate': 20240219,
 'convEndDate': 20290810,
 'firstConvPrice': 15.29,
 'bondConvPrice': 12.4,
 'rateClause': '',
 'forceRedeemTradeDate': 0,
 'forceRedeemConvDate': 0,
 'forceRedeemPrice': 0.0,
 'triggerForceRedeemPrice': 16.12,
 'triggerRepurchasePrice': 8.68,
 'expireRedeemPrice': 0.0,
 'analYTM': 0.0,
 'analPTM': 5.0877,
 'analAccruedinterst': 0.0,
 'analStrbvalue': 88.91469,
 'analStrbpremium': 42.950507,
 'analConvvalue': 114.48,
 'analConvpremiumratio': 11.0272,
 'analDuration': 4.992876,
 'analConvexity': 31.006595,
 'infoProv

In [56]:
cb_info['level']

'A+'

In [58]:
def CB_Credit(row):
    TS_CODE = str(row['ts_code'])
    CB_info = xtdata.get_cb_info(TS_CODE)
    return CB_info['level']
CB_INFO['Credit'] = CB_INFO.apply(CB_Credit, axis=1)
CB_INFO

,ts_code,bond_full_name,bond_short_name,stk_code,stk_short_name,maturity,par,issue_price,issue_size,remain_size,...,pay_per_year,list_date,delist_date,exchange,conv_start_date,conv_end_date,first_conv_price,conv_price,rate_clause,Credit
0,125002.SZ,万科企业股份有限公司可转换公司债券,万科转债,000002.SZ,万科A,5.0,100.0,100.0,1.500000e+09,0.0,...,1,2002-06-28,2004-04-30,SZ,2002-12-13,2007-06-12,12.10,5.850,票面利率为每年1.5%,
1,125009.SZ,中国宝安集团股份有限公司可转换公司债券,宝安转券,000009.SZ,中国宝安,3.0,1.0,NaN,5.000000e+08,0.0,...,1,1993-02-10,1996-01-01,SZ,1993-06-01,1995-12-31,25.00,19.392,3%,
2,125069.SZ,深圳华侨城控股股份有限公司可转换公司债券,侨城转债,000069.SZ,华侨城A,3.0,100.0,100.0,4.000000e+08,0.0,...,1,2004-01-16,2005-04-29,SZ,2004-07-01,2006-12-30,6.15,6.150,"本次发行的可转换公司债券按票面金额计算利息,计息起始日为可转换公司债券发行首日,票面利率第一...",
3,125301.SZ,吴江丝绸股份有限公司可转换公司债券,丝绸转债,000301.SZ,东方盛虹,5.0,100.0,100.0,2.000000e+08,0.0,...,1,1998-09-15,2003-08-28,SZ,2000-05-29,2003-08-27,4.10,4.100,"可转债按票面金额由1998年8月28日起开始计算利息,可转债发行首年票面利率为1.00%,以...",
4,126301.SZ,吴江丝绸股份有限公司可转换公司债券,丝绸转2,000301.SZ,东方盛虹,5.0,100.0,100.0,8.000000e+08,0.0,...,1,2002-09-24,2006-09-18,SZ,2003-03-10,2007-09-08,8.78,3.000,本次可转换公司债券的年利率为1.8%。,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1007,118048.SH,广东利扬芯片测试股份有限公司向不特定对象发行可转换公司债券,利扬转债,688135.SH,利扬芯片,6.0,100.0,100.0,5.200000e+08,520000000.0,...,1,None,None,SH,2025-01-08,2030-07-01,16.13,16.130,"20240702-20250701,票面利率:0.2%;20250702-20260701,...",A+
1008,123241.SZ,深圳欧陆通电子股份有限公司向不特定对象发行可转换公司债券,欧通转债,300870.SZ,欧陆通,6.0,100.0,100.0,6.445300e+08,644530000.0,...,1,None,None,SZ,2025-01-11,2030-07-04,44.86,44.860,"20240705-20250704,票面利率:0.10%;20250705-20260704...",AA-
1009,123242.SZ,广州市聚赛龙工程塑料股份有限公司向不特定对象发行可转换公司债券,赛龙转债,301131.SZ,聚赛龙,6.0,100.0,100.0,2.500000e+08,250000000.0,...,1,None,None,SZ,2025-01-12,2030-07-07,36.81,36.810,"20240708-20250707,票面利率:0.30%;20250708-20260707...",A+
1010,123243.SZ,浙江严牌过滤技术股份有限公司向不特定对象发行可转换公司债券,严牌转债,301081.SZ,严牌股份,6.0,100.0,100.0,4.678900e+08,467890000.0,...,1,None,None,SZ,2025-01-16,2030-07-09,7.58,7.580,"20240710-20250709,票面利率:0.20%;20250710-20260709...",A+


In [62]:
CB_level = CB_INFO[['list_date', 'delist_date', 'conv_start_date', 'conv_end_date', 'ts_code', 'Credit']]